In [2]:
from IPython.display import clear_output 
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import torch
from torchvision.utils import make_grid
from torchvision.io import read_image
# from torchvision import models, transforms
# import sys
# import os
from PIL import Image
import torch
# import torch.functional as F
import numpy as np
clear_output()
# import glob
from utils import *
from datetime import date
from torch.optim import lr_scheduler
import torchvision
from constants import mean, std
from PapilledemaLoader import OHTSDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score

In [4]:
data_dir = '/mnt/c/Users/PCM/Dropbox/OHTS/'
image_datasets = {x: OHTSDataset(data_dir=data_dir, phase=x) for x in ['train', 'val', 'test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val',  'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device, class_names)

cuda:0 ['glaucoma', 'normal']


In [5]:
pretrain_mode = 'ConPro' #'

# Load from ConPrO weights
bestsmodel = SeverityModel().to(device)
state_dict = torch.load('./pretrained/OHTS-smodel50-wofreeze-2024-03-12.pt')
bestsmodel.load_state_dict(state_dict)
bestsmodel.bestsimese50simclr.cnn1.fc = torch.nn.Sequential(*(list(bestsmodel.bestsimese50simclr.cnn1.fc)+list(bestsmodel.bestsimese50simclr.cnn1.fc2)))
torch.save(bestsmodel.bestsimese50simclr.cnn1.state_dict(), f'./pretrained/OHTS-pretrained-resnet50-ConPro.pt')

# Load from SupCon-2 weights
smodel = SeverityModel(path2pretrained='./pretrained/OHTS-contrastive50-2024-03-12.pt').to(device)
torch.save(smodel.bestsimese50simclr.cnn1.state_dict(), f'./pretrained/OHTS-pretrained-resnet50-SupCon.pt')

# Load from Simclr weights
simclrweight = get_feature_extractor(feature_extractor='resnet50', cotrain=False, simclr='/mnt/c/Users/PCM/Dropbox/pretrained/SimCLR/checkpoint_0050.pth.tar')
torch.save(simclrweight.state_dict(), f'./pretrained/OHTS-pretrained-resnet50-SimCLR.pt')

load simclr resnet
load simclr resnet
load simclr resnet


In [6]:
pretrain_mode = 'ConPro'

clf_model = models.resnet50(weights='ResNet50_Weights.DEFAULT')
if(pretrain_mode != 'ImageNet'):
    clf_model.load_state_dict(torch.load(f'./pretrained/OHTS-pretrained-resnet50-{pretrain_mode}.pt'), strict=False)
clf_model.fc = nn.Sequential(torch.nn.Linear(2048, 1000),
                                torch.nn.ReLU(),
                                torch.nn.Dropout(0.1),
                                torch.nn.Linear(1000, 256),
                                torch.nn.ReLU(),
                                torch.nn.Dropout(0.1),
                                torch.nn.Linear(256, len(class_names)))

class_weights= [0.31,       1.72222222, 1.29166667, 1.19230769, 2.58333333, 5.16666667]
# print(class_weights)
clf_model = clf_model.to(device)

momentum = 0.9
lr = 0.01
optimizer_ft = optim.SGD([{'params': clf_model.fc[:].parameters()}], lr=lr, momentum=momentum)
# class_weights=torch.tensor(class_weights,dtype=torch.float).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.5)

for param in clf_model.parameters():
    param.requires_grad = False
for param in clf_model.fc[:].parameters():
    param.requires_grad = True

valaccmax = 0
for e in range(50):
    training_acc = 0
    val_acc = 0
    training_loss_test = 0.0

    for inputs, labels in dataloaders['train']:
        clf_model.train()
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer_ft.zero_grad()

        outputs = clf_model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer_ft.step()
        training_loss_test += loss.item() * inputs.size(0)
        training_acc += torch.sum(preds == labels.data)

    for inputs, labels in dataloaders['val']:
        clf_model.eval()
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = clf_model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = loss_fn(outputs, labels)

        val_acc += torch.sum(preds == labels.data)

    if(e > 8 and val_acc >= valaccmax):
        valaccmax = val_acc
        torch.save(clf_model.state_dict(), f'./pretrained/OHTS-classfication-siamese50-{pretrain_mode}.pt')

    scheduler.step()
    print(f"E{e} With LR {optimizer_ft.param_groups[0]['lr']} training acc: ", training_acc.detach().cpu().numpy() / dataset_sizes['train'], "Val acc: ", val_acc.detach().cpu().numpy() / dataset_sizes['val'], "traning loss: ", training_loss_test / dataset_sizes['train'])

KeyboardInterrupt: 

In [7]:
bestmodel = models.resnet50(weights='ResNet50_Weights.DEFAULT')#get_feature_extractor(feature_extractor='resnet50', cotrain=False)#, simclr='/mnt/c/Users/PCM/Documents/GitHub/pseudopapill/SimCLR/runs/Oct29_21-00-13_DESKTOP-404G4HS/checkpoint_95_29102023.pth.tar')
bestmodel.fc = nn.Sequential(torch.nn.Linear(2048, 1000),
                                torch.nn.ReLU(),
                                torch.nn.Dropout(0.1),
                                torch.nn.Linear(1000, 256),
                                torch.nn.ReLU(),
                                torch.nn.Dropout(0.1),
                                torch.nn.Linear(256, len(class_names)))
bestmodel.load_state_dict(torch.load(f'./pretrained/OHTS-classfication-siamese50-{pretrain_mode}.pt'))
bestmodel.to(device)

test_acc = 0
predlist = []
labelist = []
problist = []
# test_embeddings = torch.zeros((0, 2048))
sedis = 0
# fextractor = torch.nn.Sequential(*(list(clf_model.children())[:-1]))

for inputs, labels in dataloaders['test']:
    bestmodel.eval()
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = bestmodel(inputs)
        # emb = fextractor(inputs)
        _, preds = torch.max(outputs, 1)
        # loss = loss_fn(outputs, labels)
        sedis = sedis + torch.sum(torch.exp(torch.abs(labels - torch.max(outputs, 1)[1])))
    problist.append(outputs[:,1].detach().cpu().numpy())
    labelist.append(labels.detach().cpu().numpy()*1)
    predlist.append(preds.detach().cpu().numpy())
    # test_embeddings  = torch.cat((test_embeddings, emb.detach().cpu().flatten().unsqueeze(0)), axis=0)
    test_acc += torch.sum(preds == labels.data)

In [10]:
labelist = np.concatenate(labelist).ravel()
problist = np.concatenate(problist).ravel()
predlist = np.concatenate(predlist).ravel()

In [11]:
print(classification_report(labelist, predlist, digits=3))

              precision    recall  f1-score   support

           0      0.791     0.884     0.835      1460
           1      0.626     0.453     0.526       625

    accuracy                          0.755      2085
   macro avg      0.708     0.669     0.680      2085
weighted avg      0.741     0.755     0.742      2085

